# Superhuman genes data filtering notebook

The purpose of this notebook to parse data provided by curator

In [4]:
import polars as pl
from pathlib import *
import sqlite3

### Setting up paths

In [5]:
base = Path(".")
data = base / "data"
input = data / "input"
output = data / "output"

In [6]:
superhuman_path = input / "superhuman.tsv"
superhuman_db = pl.read_csv(str(superhuman_path), sep='\t')
superhuman_db.head()

rsID,Gene,Polymorphism,Location 38,Ref allele,Alt allele,Genotype,Zygosity,Superability,Adverse effects,References
str,str,str,str,str,str,str,str,str,str,str
"""rs121908668""","""LRP5""","""G171V/+""","""11:68357673""","""G""","""T""",null,null,"""Extra strong b...","""Osteosclerosis...","""https://pubmed..."
"""rs12105165""","""MSTN""","""-/-""","""2:190055744""","""G""","""A, C, T""",null,null,"""Large, lean mu...","""Not reported""","""https://medlin..."
"""rs374256136""","""MSTN""","""-/-""","""2:190057641""","""G""","""A""",null,null,"""Large, lean mu...","""Not reported""","""https://medlin..."
"""rs3187415""","""MSTN""","""-/-""","""2:190056055""","""C""","""G,T""",null,null,"""Large, lean mu...","""Not reported""","""https://medlin..."
"""rs1805085""","""MSTN""","""-/-""","""2:190062434""","""C""","""T""",null,null,"""Large, lean mu...","""Not reported""","""https://medlin..."


Dividing column for Alt allele into separate rows

In [7]:
superhuman = (
    superhuman_db
    .with_columns(
        [
            (pl.col("Alt allele").str.replace(' ', '')
                            .str.split(',')),
            (pl.col("Zygosity").fill_null('both'))
        ]
    )
    .pipe(
        lambda df:df.rename({oldCol:oldCol.lower().replace(' ','_') for oldCol in df.columns})
    )
)

superhuman.head()

rsid,gene,polymorphism,location_38,ref_allele,alt_allele,genotype,zygosity,superability,adverse_effects,references
str,str,str,str,str,list[str],str,str,str,str,str
"""rs121908668""","""LRP5""","""G171V/+""","""11:68357673""","""G""","[""T""]",null,"""both""","""Extra strong b...","""Osteosclerosis...","""https://pubmed..."
"""rs12105165""","""MSTN""","""-/-""","""2:190055744""","""G""","[""A"", ""C"", "" T""]",null,"""both""","""Large, lean mu...","""Not reported""","""https://medlin..."
"""rs374256136""","""MSTN""","""-/-""","""2:190057641""","""G""","[""A""]",null,"""both""","""Large, lean mu...","""Not reported""","""https://medlin..."
"""rs3187415""","""MSTN""","""-/-""","""2:190056055""","""C""","[""G"", ""T""]",null,"""both""","""Large, lean mu...","""Not reported""","""https://medlin..."
"""rs1805085""","""MSTN""","""-/-""","""2:190062434""","""C""","[""T""]",null,"""both""","""Large, lean mu...","""Not reported""","""https://medlin..."


Preparing names of columns for sqlite db

In [8]:
superhuman_pandas = superhuman.explode('alt_allele').to_pandas()

Clearing output

In [12]:
import shutil
if output.exists():
    shutil.rmtree(output)
output.mkdir()

Creating sqlite database

In [13]:
sqlite_db_path = output / 'superhuman.sqlite'

sqlite_con = sqlite3.connect(sqlite_db_path)
(
    superhuman_pandas
    .sort_values("rsid")
    .to_sql('superhuman', sqlite_con, if_exists='replace', index=False)
)


1243

In [14]:
sqlite_con.close()